In [1]:
import sys
sys.path.append("..")

path = !cd .. && pwd
path = path[0]

import torch 
import argparse
from assembler import get_base_model
from utils.config import get_config_base_model, get_config_hash
from pytorch_lightning import Trainer
from utils.dataloader import dataloader

config = get_config_base_model(f"{path}/configs/celeba/gan/dcgan.yaml")
vae = get_base_model(config, "celeba", "training")

# Load data
dm = dataloader(config)

TypeError: __init__() missing 2 required positional arguments: 'feature_maps' and 'image_channels'

In [3]:
# Run Training Loop
trainer= Trainer(gpus = config["trainer_params"]["gpus"],
        max_epochs = config["trainer_params"]["max_epochs"])
trainer.fit(vae, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/home/mingliang/anaconda3/envs/dgflowenv/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:99: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  rank_zero_warn(f'you passed in a {loader_name} but have no {step_name}. Skipping {stage} loop')


celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


100%|██████████| 2.02M/2.02M [00:00<00:00, 45.1MB/s]


100%|██████████| 1.54M/1.54M [00:00<00:00, 62.8MB/s]


100%|██████████| 466k/466k [00:00<00:00, 67.8MB/s]


100%|██████████| 2.07M/2.07M [00:00<00:00, 50.9MB/s]


Done!


FileNotFoundError: [Errno 2] No such file or directory: 'data/list_eval_partition.csv'